In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
os.getcwd()

'/home/bk_anupam/code/ML/KaggleChallenges/AUDIO/MusicClassification'

In [3]:
NUM_CLASSES = 19
SUB_PATH = "./data/submission/"

In [4]:
df_effnet_b4 = pd.read_csv(SUB_PATH + "effnet_b4_51536.csv")
df_resnet50 = pd.read_csv(SUB_PATH + "resnet50_51417.csv")
df_resnext50 = pd.read_csv(SUB_PATH + "resnext50_51896.csv")
df_resnext50_5folds = pd.read_csv(SUB_PATH + "submission_effnet_b4_44100_52574.csv")
df_resnext50_5folds = pd.read_csv(SUB_PATH + "submission_resnext50_44100_52335.csv")
df_pred_list = [df_effnet_b4, df_resnet50, df_resnext50, df_resnext50_5folds]

In [5]:
def get_ensemble_pred(song_id, df_pred_list):
    preds = np.zeros(NUM_CLASSES)
    for df in df_pred_list:
        df_pred = df[df.song_id == song_id].genre_id        
        preds[df_pred] += 1
    return np.argmax(preds)

In [6]:
df_effnet_b4["ensemble_genre_id"] = df_effnet_b4.song_id.apply(
    lambda song_id: get_ensemble_pred(song_id, df_pred_list)
)

In [7]:
df_effnet_b4[df_effnet_b4.genre_id != df_effnet_b4.ensemble_genre_id]

,song_id,genre_id,ensemble_genre_id
8,32,3,2
14,57,11,8
19,79,5,1
37,148,1,2
38,153,13,3
...,...,...,...
5044,24862,2,1
5045,24864,7,4
5054,24906,3,0
5071,24986,7,3


In [8]:
df_ensemble_pred = df_effnet_b4[["song_id", "ensemble_genre_id"]]
df_ensemble_pred = df_ensemble_pred.rename({"ensemble_genre_id": "genre_id"}, axis=1)
df_ensemble_pred

,song_id,genre_id
0,3,4
1,6,11
2,8,1
3,11,2
4,17,5
...,...,...
5073,24988,1
5074,24991,1
5075,24994,0
5076,22612,0


In [9]:
df_ensemble_pred.to_csv(SUB_PATH + "ensemble_preds.csv", index=False)